# Model Discrimination for  R32/emimTF2N system

EoS: PR vs. SRK

Parameter T dependence: Linear

N (total fitting parameters): 6

## Import Functions


In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

import scipy.stats as stats

In [2]:
params_PR = pd.read_csv('../Params/PR_params_quadTdep.csv',header=None)

In [3]:
params_PR

,0
0,0.321298
1,0.842982
2,-0.553211
3,-0.610086
4,0.186268
5,-0.393545
6,0.000000
7,0.000000


## Make model functions

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
def calc_PRmod_P(x_HFC,T):
    '''
    Define function to calculate the PR 4 parameter model pressure given a composition and temperature
    
    Inputs:
    x_HFC - HFC compositions
    T - temperature to run
    
    Output: 
    P: pressure
    
    '''
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    # Configuration
    from hfc32_emimtf2n_PR import configuration as config_PR

    m.fs.properties = GenericParameterBlock(default=config_PR)

    m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

    m.fs.F101.inlet.flow_mol.fix(1)
    m.fs.F101.inlet.temperature.fix(323.15)
    m.fs.F101.inlet.pressure.fix(399800)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)
    
    # Model params
    params_PR = pd.read_csv('../Params/PR_params_linTdep.csv',header=None)
    
    m.fs.properties.PR_kappa_A['R32','emimTf2N'] = float(params_PR.iloc[0])
    m.fs.properties.PR_kappa_A['emimTf2N','R32'] = float(params_PR.iloc[1])
    m.fs.properties.PR_kappa_B['R32','emimTf2N'] = float(params_PR.iloc[2])
    m.fs.properties.PR_kappa_B['emimTf2N','R32'] = float(params_PR.iloc[3])
    m.fs.properties.PR_kappa_C['R32','emimTf2N'] = float(params_PR.iloc[2])
    m.fs.properties.PR_kappa_C['emimTf2N','R32'] = float(params_PR.iloc[3])
    
    # Initialize the flash unit
    m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    # m.fs.F101.inlet.pressure.unfix()
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

    m.fs.liq = Param(mutable=True,default=0.040)
    m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)
    
    # Calc pressure
    x_IL = 1-x_HFC
    if 0.05 <= x_HFC <= 0.09:
        guess_P = 90000
    elif 0.09 < x_HFC <= 0.15:
        guess_P = 100100
    elif 0.15 < x_HFC <= 3:
        guess_P = 250000
    else:
        guess_P = 400000
    
    m.fs.liq = x_IL
    m.fs.F101.inlet.temperature.fix(T)
    m.fs.F101.inlet.pressure.fix(guess_P)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(x_HFC+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(x_HFC+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(T))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            x_HFC_final = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P_final = value(m.fs.F101.vap_outlet.pressure[0])
        else:
            print('Infeasible Solution')
            x_HFC_final = np.nan
            P_final = np.nan
            
    except ValueError:
        x_HFC_final = np.nan
        P_final = np.nan
        print('Solver Error')
    
    return P_final

In [ ]:
def calc_SRKmod_P(x_HFC,T):
    '''
    Define function to calculate the SRK 4 parameter model pressure given a composition and temperature
    
    Inputs:
    x_HFC - HFC compositions
    T - temperature to run
    
    Output: 
    P: pressure
    
    '''
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    # Configuration
    from hfc32_emimtf2n_SRK import configuration as config_SRK

    m.fs.properties = GenericParameterBlock(default=config_SRK)

    m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

    m.fs.F101.inlet.flow_mol.fix(1)
    m.fs.F101.inlet.temperature.fix(323.15)
    m.fs.F101.inlet.pressure.fix(399800)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)
    
    # Model params
    params_SRK = pd.read_csv('../Params/SRK_params_linTdep.csv',header=None)
    
    m.fs.properties.SRK_kappa_A['R32','emimTf2N'] = float(params_SRK.iloc[0])
    m.fs.properties.SRK_kappa_A['emimTf2N','R32'] = float(params_SRK.iloc[1])
    m.fs.properties.SRK_kappa_B['R32','emimTf2N'] = float(params_SRK.iloc[2])
    m.fs.properties.SRK_kappa_B['emimTf2N','R32'] = float(params_SRK.iloc[3])
    
    # Initialize the flash unit
    m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    # m.fs.F101.inlet.pressure.unfix()
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

    m.fs.liq = Param(mutable=True,default=0.040)
    m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)
    
    # Calc pressure
    x_IL = 1-x_HFC
    if 0.05 <= x_HFC <= 0.09:
        guess_P = 90000
    elif 0.09 < x_HFC <= 0.15:
        guess_P = 100100
    elif 0.15 < x_HFC <= 3:
        guess_P = 250000
    else:
        guess_P = 400000
    
    
    m.fs.liq = x_IL
    m.fs.F101.inlet.temperature.fix(T)
    m.fs.F101.inlet.pressure.fix(guess_P)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(x_HFC+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(x_HFC+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(T))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            x_HFC_final = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P_final = value(m.fs.F101.vap_outlet.pressure[0])
        else:
            print('Infeasible Solution')
            x_HFC_final = np.nan
            P_final = np.nan
            
    except ValueError:
        x_HFC_final = np.nan
        P_final = np.nan
        print('Solver Error')
    
    return P_final

In [ ]:
def error(params):
    '''
    Function which calculates the squared error between two models
 
    Inputs:
    params:
        x_HFC - HFC compositions
        T - temperature to run
    
    Ouputs:
    error - squared error between two models, made negative
    '''
    
    x_HFC = params[0]
    T = params[1]
    
    print('Trying x_HFC:',x_HFC)
    print('Trying T:',T)
    
    P_PR = calc_PRmod_P(x_HFC,T)
    
    P_SRK = calc_SRKmod_P(x_HFC,T)
    
    error = (P_PR - P_SRK)**2
    
    print('error:',error)
    
    return error*(-1)

## Setup Optimization Problem for Model Discrimination

In [ ]:
var_guess = (0.1,280)
bnds = ((0.05,0.8),(275,360))
results = scipy.optimize.minimize(error,var_guess,bounds=bnds,tol=1E-4)

In [ ]:
results

In [ ]:
var_guess = (0.1,280)
bnds = ((0.0755,0.8),(272.5,360))
results = scipy.optimize.minimize(error,var_guess,bounds=bnds,tol=1E-4)

In [ ]:
results 

In [ ]:
var_guess = (0.6,280)
bnds = ((0.15,0.8),(265,360))
results = scipy.optimize.minimize(error,var_guess,bounds=bnds,tol=1E-4)

In [ ]:
results